In [ ]:
# Zero-shot LLM solo con término y dominio 

import os
import pandas as pd
from transformers import pipeline
from tqdm import tqdm

# ------------------ CONFIG ------------------
HF_TOKEN = "hf_token"  # tu token de Hugging Face
os.environ["HUGGINGFACEHUB_API_TOKEN"] = HF_TOKEN

MODEL_LLM = "mistralai/Mistral-7B-v0.1"

DATA_PATH = "/home/jovyan/lrec_2026/data_10/fin_institution/glossary_fin_institution_info_semantica_es.csv"
OUTPUT_PATH = "/home/jovyan/lrec_2026/data_10/fin_institution/migration_fin_institution_zero_shot_mistral.csv"

LIMIT_TERMS = 100
MANUAL_DOMAIN = "migración" 

# ------------------ CARGA CSV ------------------
print("📥 Cargando CSV...")
df = pd.read_csv(DATA_PATH, sep=";", quotechar='"', engine="python", on_bad_lines="skip")

if "term" not in df.columns:
    raise ValueError("❌ El CSV debe tener la columna 'term'.")

df = df.head(LIMIT_TERMS)
df["term"] = df["term"].astype(str).fillna("")

# ------------------ LLM ------------------
print("🔹 Cargando modelo Llama 3.1...")
llm_pipeline = pipeline(
    "text-generation",
    model=MODEL_LLM,
    tokenizer=MODEL_LLM,
    device_map="auto",
    torch_dtype="auto",
    max_new_tokens=128,
    temperature=0.2,
    repetition_penalty=1.1
)
print("✅ Modelo cargado correctamente")

# ------------------ GENERAR DEFINICIONES ------------------
def generate_definition(term, domain):
    """
    Zero-shot definition generation.
    - Usa solo el término y el dominio manual
    - Garantiza UNA DEFINICIÓN corta, en español y sin saltos de línea
    """
    prompt = (
        f"Eres un experto en terminología jurídica.\n"
        f"Redacta UNA DEFINICIÓN precisa y concisa en ESPAÑOL para el término '{term}'.\n"
        f"El término pertenece al dominio: '{domain}'.\n"
        f"No agregues explicaciones adicionales ni inventes información.\n"
        f"Definición:"
    )

    output = llm_pipeline(prompt)[0]["generated_text"]
    definition = output.split("Definición:")[-1].strip()
    definition = " ".join(definition.split())
    if len(definition) > 150:
        definition = definition[:150].rstrip(".,;") + "."
    return definition

# ------------------ PROCESAMIENTO ------------------
definitions = []
for _, row in tqdm(df.iterrows(), total=len(df), desc="definiciones zero-shot"):
    try:
        definition = generate_definition(row["term"], MANUAL_DOMAIN)
    except Exception as e:
        definition = f"ERROR: {e}"
    definitions.append(definition)

df["definition"] = definitions
df.to_csv(OUTPUT_PATH, sep=";", index=False)
print(f"\n✅ Archivo final guardado en: {OUTPUT_PATH}")
print(f"📘 Términos procesados: {len(df)}")
